# Testing the computation time for LJ Pot

This jupyter notebook is intented to test the computation time of 
different implementations for the Lennard Jones potential with an  
S1 switch function.  
Generally it should also be able to handle particle types. <br>



In [ ]:
import math
import torch
import numpy as np
from numba import vectorize, cuda, njit, jit
import torch
import ewald_summation as es

In [ ]:
%load_ext line_profiler

In [ ]:
N = 1000

configuration = np.random.uniform(0, 4, (N, 3))
sigma = np.array([1] * N)
sigma_arr = 0.5 * (sigma[:, None] + sigma)
sigma_arr_6 = sigma_arr**6
sigma_arr_12 = sigma_arr**12
epsilon = np.array([1] * N)
epsilon_arr = np.sqrt(epsilon[:, None] * epsilon)
cutoff_lj = 3.5
switch_width_lj = 1
switch_start_lj = 2.5

# Loops

simple implementation with loops  


In [ ]:
def lj_potential_pairwise(distance, sigma_lj, epsilon_lj):
    if(distance <= 0 or distance > cutoff_lj):
        return 0.
    else:
        inv_dist = sigma_lj / distance
        inv_dist2 = inv_dist * inv_dist
        inv_dist4 = inv_dist2 * inv_dist2
        inv_dist6 = inv_dist2 * inv_dist4
        phi_LJ = 4. * epsilon_lj * inv_dist6 * (inv_dist6 - 1.)
        if(distance <= cutoff_lj - switch_width_lj):
            return phi_LJ
        else:
            t = (distance - cutoff_lj) / switch_width_lj
            switch = t * t * (3. + 2. * t)
            return phi_LJ * switch
        
def lj_potential_loops(x):
    output = np.zeros(len(x))
    for i in range(len(x)):
        potential = 0
        for j in range(i, len(x)):
            sigma_lj = sigma_arr[i, j]
            epsilon_lj = epsilon_arr[i, j]
            distance = np.linalg.norm(x[i, :] - x[j, :])
            potential += lj_potential_pairwise(distance,sigma_lj, epsilon_lj)
        output[i] = potential
    return output


In [ ]:
x = np.random.uniform(0, 4, (100, 3))
np.testing.assert_allclose(np.sum(lj_potential_loops(x)), es.potentials.lj.lj_potential_total(x))

In [ ]:
%lprun -f lj_potential_loops lj_potential_loops(configuration)

In [ ]:
timing_loops = %timeit lj_potential_loops(configuration)

# Numpy
Using np.apply_along_axis, which allows for convienient implementation

In [ ]:
def potential_along_axis(x):
        # potenital w/o switch
        if x[0] > 0 and x[0] <= cutoff_lj - switch_width_lj:
            x[0] = 4 * x[2] * x[1]**6 * (x[1]**6 / x[0]**12 - 1 / x[0]**6)
        # potential with switch
        elif x[0] > cutoff_lj - switch_width_lj and x[0] <= cutoff_lj:
            t = (x[0] - cutoff_lj) / (cutoff_lj - cutoff_lj - switch_width_lj)
            switch = 2 * t ** 3 + 3 * t ** 2
            x[0] = switch * (4 * x[2] * x[1]**6 * (x[1]**6 / x[0]**12 - 1 / x[0]**6))
        # potential after cutoff
        elif x[0] > cutoff_lj:
            x[0] = 0
        return x[0]

def lj_potential_np_along_axis(x, sigma_arr, epsilon_arr):
    # initialize output as array with distances and corresponding sigma, epsilon along axis=2
    output = np.zeros((x.shape[0], x.shape[0], 3))
    output[:, :, 0] = np.linalg.norm(x[:, None, :] - x[None, :, :], axis=-1)
    output[:, :, 1] = sigma_arr
    output[:, :, 2] = epsilon_arr
    # calculate potentials
    output[:, :, 0] = np.apply_along_axis(potential_along_axis, 2, output)
    output = np.sum(output[:, :, 0], axis=-1)
    return output

In [ ]:
x = np.random.uniform(0, 4, (100, 3))
sigma_temp = np.array([1] * len(x))
sigma_arr_temp= 0.5 * (sigma_temp[:, None] + sigma_temp)
epsilon_temp = np.array([1] * len(x))
epsilon_arr_temp = np.sqrt(epsilon_temp[:, None] * epsilon_temp)

np.testing.assert_allclose(0.5 * np.sum(lj_potential_np_along_axis(x, sigma_arr_temp, epsilon_arr_temp)), 
                           es.potentials.lj.lj_potential_total(x))

In [ ]:
%lprun -f lj_potential_np_along_axis lj_potential_np_along_axis(configuration, sigma_arr, epsilon_arr)

In [ ]:
timings_np_apply_along_axis = %timeit lj_potential_np_along_axis(configuration, sigma_arr, epsilon_arr)

# turns out its slow
Replace apply_along_axis with a loop

In [ ]:
def potential_along_axis(x):
        # potenital w/o switch
        if x[0] > 0 and x[0] <= cutoff_lj - switch_width_lj:
            x[0] = 4 * x[2] * x[1]**6 * (x[1]**6 / x[0]**12 - 1 / x[0]**6)
        # potential with switch
        elif x[0] > cutoff_lj - switch_width_lj and x[0] <= cutoff_lj:
            t = (x[0] - cutoff_lj) / (cutoff_lj - cutoff_lj - switch_width_lj)
            switch = 2 * t ** 3 + 3 * t ** 2
            x[0] = switch * (4 * x[2] * x[1]**6 * (x[1]**6 / x[0]**12 - 1 / x[0]**6))
        # potential after cutoff
        elif x[0] > cutoff_lj:
            x[0] = 0
        return x[0]

def lj_potential_np_along_axis_replace_loop(x, sigma_arr, epsilon_arr):
    # initialize output as array with distances and corresponding sigma, epsilon along axis=2
    output = np.zeros((x.shape[0], x.shape[0], 3))
    output[:, :, 0] = np.linalg.norm(x[:, None, :] - x[None, :, :], axis=-1)
    output[:, :, 1] = sigma_arr
    output[:, :, 2] = epsilon_arr
    # calculate potentials
    for i in range(len(output)):
        for j in range(i, len(output)):
            output[i, j, 0] = potential_along_axis(output[i, j, :])
    output = np.sum(output[:, :, 0], axis=-1)
    return output

In [ ]:
x = np.random.uniform(0, 4, (100, 3))
sigma_temp = np.array([1] * len(x))
sigma_arr_temp= 0.5 * (sigma_temp[:, None] + sigma_temp)
epsilon_temp = np.array([1] * len(x))
epsilon_arr_temp = np.sqrt(epsilon_temp[:, None] * epsilon_temp)

np.testing.assert_allclose(np.sum(lj_potential_np_along_axis_replace_loop(x, sigma_arr_temp, epsilon_arr_temp)), 
                           es.potentials.lj.lj_potential_total(x))

In [ ]:
%lprun -f lj_potential_np_along_axis_replace_loop lj_potential_np_along_axis_replace_loop(configuration, sigma_arr, epsilon_arr)

In [ ]:
timings_np_apply_along_axis_replace_loop = %timeit lj_potential_np_along_axis_replace_loop(configuration, sigma_arr, epsilon_arr)

# Numpy.piecewise

Allows for somewhat convenient implementation but does not handle particle types easily

In [ ]:
def lj_potential_np_piecewise(x, sigma, epsilon):
        distances = np.linalg.norm(x[:, None, :] - x[None, :, :], axis=-1)

        # potential_pairwise
        def p1(d):
            sigma6 = sigma ** 6
            potential = 4 * epsilon * sigma6 * (sigma6 / d ** 12 - 1 / d ** 6)
            return potential

        # potential_pairwise with switch function smoothstep S1
        def p2(d):
            t = (d - cutoff_lj) / (cutoff_lj - switch_start_lj)
            switch_function = t * t * (3. + 2. * t)
            sigma6 = sigma ** 6
            potential = 4 * epsilon * sigma6 * (sigma6 / d ** 12 - 1 / d ** 6)
            return potential * switch_function

        # piecewise function for Lennard Jones Potential
        def p12(d):
            output = np.piecewise(d, [d <= 0,
                                 (0 < d) & (d < switch_start_lj),
                                 (switch_start_lj <= d) & (d < cutoff_lj),
                                 cutoff_lj <= d],
                                 [0, p1, p2,0]
                                 )
            return output
        
         # sum potentials for every particle
        potential = np.sum(p12(distances), axis=-1)
        return potential

In [ ]:
sigma_temp = 1
epsilon_temp = 1
x = np.random.uniform(0, 4, (100, 3))
np.testing.assert_allclose(0.5 * np.sum(lj_potential_np_piecewise(x, sigma_temp, epsilon_temp)),
                           es.potentials.lj.lj_potential_total(x))

In [ ]:
%lprun -f lj_potential_np_piecewise lj_potential_np_piecewise(configuration, sigma_temp, epsilon_temp)

In [ ]:
timings_np_piecewise = %timeit lj_potential_np_piecewise(configuration, 1, 1)

# Numpy masking


In [ ]:
def lj_potential_numpy(x, sigma_arr, sigma_6_arr, sigma_12_arr, epsilon_arr):
    output = np.zeros(len(x))
    # get distances
    d = np.linalg.norm(x[:, None, :] - x[None, :, :], axis=-1)
    # get masks
    mask1 = (d > switch_start_lj * sigma_arr) | (d <= 0)
    mask2 = (d < switch_start_lj * sigma_arr) | (d > cutoff_lj * sigma_arr)
    # mask distances and arrays for mixed constants
    d_masked1 = np.ma.masked_where(mask1, d)
    d_masked2 = np.ma.masked_where(mask2, d)
    sigma_arr_6_masked1 = np.ma.masked_where(mask1, sigma_6_arr)
    sigma_arr_12_masked1 = np.ma.masked_where(mask1, sigma_12_arr)
    sigma_arr_6_masked2 = np.ma.masked_where(mask2, sigma_6_arr)
    sigma_arr_12_masked2 = np.ma.masked_where(mask2, sigma_12_arr)
    epsilon_arr_masked1 = np.ma.masked_where(mask1, epsilon_arr)
    epsilon_arr_masked2 = np.ma.masked_where(mask2, epsilon_arr)
    # calculate potential
    output = (np.array(4 * epsilon_arr_masked1 
             * (sigma_arr_12_masked1 / d_masked1**12 - sigma_arr_6_masked1 / d_masked1**6))
             + np.array(4 * epsilon_arr_masked2 
             * (sigma_arr_12_masked2 / d_masked2**12 - sigma_arr_6_masked2 / d_masked2**6)))
    return np.sum(output, axis=-1)


In [ ]:
x = np.random.uniform(0, 4, (100, 3))
sigma_temp = np.array([1] * len(x))
sigma_arr_temp = 0.5 * (sigma_temp[:, None] + sigma_temp)
sigma_arr_6_temp = sigma_arr_temp**6
sigma_arr_12_temp = sigma_arr_temp**12
epsilon_temp = np.array([1] * len(x))
epsilon_arr_temp = np.sqrt(epsilon_temp[:, None] * epsilon_temp)

np.testing.assert_allclose(0.5 * np.sum(lj_potential_numpy(x, sigma_arr_temp, sigma_arr_6_temp, sigma_arr_12_temp, 
                           epsilon_arr_temp)), es.potentials.lj.lj_potential_total(x), rtol=1E-4)

In [ ]:
%lprun -f lj_potential_numpy lj_potential_numpy(configuration, sigma_arr, sigma_arr_6, sigma_arr_12, epsilon_arr)

In [ ]:
timing_numpy = %timeit lj_potential_numpy(configuration, sigma_arr, sigma_arr_6, sigma_arr_12, epsilon_arr)

# Masking with Pytorch

In [ ]:
def lj_potential_pytorch(x, sigma_arr, sigma_6_arr, sigma_12_arr, epsilon_arr):
    # get distances, init tensors
    x = torch.Tensor(x).double()
    d = torch.norm(x[:, None, :] - x[None, :, :], dim=-1)
    sigma_arr = torch.Tensor(sigma_arr).double()
    sigma_6_arr = torch.Tensor(sigma_6_arr).double()
    epsilon_arr = torch.Tensor(epsilon_arr).double()
    # get masks
    mask1 = ((d > 0) & (d < 2.5 * sigma_arr))
    mask2 = ((d > 2.5 * sigma_arr) & (d < 3.5 * sigma_arr))
    # init output and caculate potential
    output = torch.tensor((), dtype=torch.float64).new_zeros(d.size())
    output[mask1] = (4 * epsilon_arr[mask1] * sigma_6_arr[mask1] 
                     * (sigma_6_arr[mask1] / d[mask1]**12 - sigma_6_arr[mask1] / d[mask1]**6))
    output[mask2] = (4 * epsilon_arr[mask2] * sigma_6_arr[mask2]
                     * (sigma_6_arr[mask2] / d[mask2]**12 - sigma_6_arr[mask2] / d[mask2]**6))
    output = torch.sum(output, dim=-1)
    return output
    

In [ ]:
x = np.random.uniform(0, 4, (100, 3))
sigma_temp = np.array([1] * len(x))
sigma_arr_temp = 0.5 * (sigma_temp[:, None] + sigma_temp)
sigma_arr_6_temp = sigma_arr_temp**6
sigma_arr_12_temp = sigma_arr_temp**12
epsilon_temp = np.array([1] * len(x))
epsilon_arr_temp = np.sqrt(epsilon_temp[:, None] * epsilon_temp)

np.testing.assert_allclose(0.5 * torch.sum(lj_potential_pytorch(x, sigma_arr_temp, sigma_arr_6_temp, sigma_arr_12_temp, 
                            epsilon_arr_temp)), es.potentials.lj.lj_potential_total(x), rtol=1E-5)

In [ ]:
%lprun -f lj_potential_pytorch lj_potential_pytorch(configuration, sigma_arr, sigma_arr_6, sigma_arr_12, epsilon_arr)

In [ ]:
timing_pytorch = %timeit lj_potential_pytorch(configuration, sigma_arr, sigma_arr_6, sigma_arr_12, epsilon_arr)

# Try it with CUDA
right now it floods the GPU with data, that should probably be changed

In [ ]:
def lj_potential_pytorch_cuda(x, sigma_arr, sigma_6_arr, sigma_12_arr, epsilon_arr):
    # get distances, init tensors
    x = torch.Tensor(x).double().cuda()
    d = torch.norm(x[:, None, :] - x[None, :, :], dim=-1).cuda()
    sigma_arr = torch.Tensor(sigma_arr).double().cuda()
    sigma_6_arr = torch.Tensor(sigma_6_arr).double().cuda()
    epsilon_arr = torch.Tensor(epsilon_arr).double().cuda()
    # get masks
    mask1 = ((d > 0) & (d < 2.5 * sigma_arr)).cuda()
    mask2 = ((d > 2.5 * sigma_arr) & (d < 3.5 * sigma_arr)).cuda()
    # init output and caculate potential
    output = torch.tensor((), dtype=torch.float64).new_zeros(d.size()).cuda()
    output[mask1] = (4 * epsilon_arr[mask1] * sigma_6_arr[mask1] 
                     * (sigma_6_arr[mask1] / d[mask1]**12 - sigma_6_arr[mask1] / d[mask1]**6))
    output[mask2] = (4 * epsilon_arr[mask2] * sigma_6_arr[mask2]
                     * (sigma_6_arr[mask2] / d[mask2]**12 - sigma_6_arr[mask2] / d[mask2]**6))
    output = torch.sum(output, dim=-1)
    return output

In [ ]:
%lprun -f lj_potential_pytorch lj_potential_pytorch_cuda(configuration, sigma_arr, sigma_arr_6, sigma_arr_12, epsilon_arr)

In [ ]:
timing_pytorch_cuda = %timeit lj_potential_pytorch_cuda(configuration, sigma_arr, sigma_arr_6, sigma_arr_12, epsilon_arr)

# Last but not least: Numba

In [ ]:
@njit
def lj_potential_pairwise(distance, sigma_lj, epsilon_lj):
    if(distance <= 0 or distance > cutoff_lj):
        return 0.
    else:
        inv_dist = sigma_lj / distance
        inv_dist2 = inv_dist * inv_dist
        inv_dist4 = inv_dist2 * inv_dist2
        inv_dist6 = inv_dist2 * inv_dist4
        phi_LJ = 4. * epsilon_lj * inv_dist6 * (inv_dist6 - 1.)
        if(distance <= cutoff_lj - switch_width_lj):
            return phi_LJ
        else:
            t = (distance - cutoff_lj) / switch_width_lj
            switch = t * t * (3. + 2. * t)
            return phi_LJ * switch

@njit
def lj_potential_numba(x):
    output = np.zeros(len(x))
    for i in range(len(x)):
        potential = 0
        for j in range(i, len(x)):
            sigma = sigma_arr[i, j]
            epsilon = epsilon_arr[i, j]
            distance = np.linalg.norm(x[i, :] - x[j, :])
            potential += lj_potential_pairwise(distance, sigma, epsilon)
        output[i] = potential
    return output

x = np.random.uniform(0, 4, (10,3))

In [ ]:
x = np.random.uniform(0, 4, (100, 3))
np.testing.assert_allclose(np.sum(lj_potential_numba(x)), es.potentials.lj.lj_potential_total(x))

In [ ]:
%lprun -f lj_potential_numba(configuration) lj_potential_numba(configuration)

In [ ]:
timing_numba = %timeit lj_potential_numba(configuration)

I tried @njit(parallelize=True) but it was much slower for some reason